In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import math
from operator import itemgetter

import nest_asyncio
nest_asyncio.apply()
import asyncio

from teeport import Teeport

## Train a RL model to tune the BTS correctors

### Grab the BTS tracking function

In [6]:
teeport = Teeport('ws://lambda-sp3:8090')

In [7]:
process = teeport.use_processor('6La3OgFJq')

### Define the BTS env

In [8]:
import gym
from gym import spaces

In [141]:
class BTSEnv(gym.Env):
    """
    Custom Environment that follows gym interface.
    This is an env where the agent must learn to tune the correctors
    to get the maximum injection efficien. 
    """
    metadata = {'render.modes': ['console']}
    # Define constants for clearer code
    H1_UP = 0
    H1_DOWN = 1
    V1_UP = 2
    V1_DOWN = 3
    H5_UP = 4
    H5_DOWN = 5
    V2_UP = 6
    V2_DOWN = 7

    def __init__(self, process, knob_step_size=0.1, max_steps=10000, verbose=False):
        super(BTSEnv, self).__init__()
        
        # The trackBTS processor
        self.process = process
        self.knob_step_size = knob_step_size
        self.max_steps = max_steps
        self.step_count = 0
        self.verbose = verbose
        
        # Initialize the agent at the middle position of the knobs
        # and the zero-offset point of dX
        self.knob_pos = 0.5 * np.ones(4)
        self.dX = 0.5 * np.ones(6)
        
        self.inj_eff = None

        # Define action and observation space
        # They must be gym.spaces objects
        # We have 4 knobs, each knob has two actions: go up and go down by 1 unit
        # 0: h1 + 0.1, 1: h1 - 0.1
        # 2: v1 + 0.1, 3: v1 - 0.1
        # 4: h5 + 0.1, 5: h5 - 0.1
        # 6: v2 + 0.1, 7: v2 - 0.1
        self.action_space = spaces.Discrete(8)
        # The observation will be the readout of the BPMs, plus the position of the 4 knobs
        # this can be described both by Discrete and Box space
        low = np.array([-100] * 18 + [0] * 4)
        high = np.array([100] * 18 + [1] * 4)
        self.observation_space = spaces.Box(low=low, high=high, shape=(22,), dtype=np.float32)
    
    def get_reward(self, quiet=False):
        X = 0.5 * np.ones(22)
        X[:6] = self.dX
        X[6] = self.knob_pos[0]  # h1
        X[10] = self.knob_pos[2]  # h5
        X[15] = self.knob_pos[1]  # v1
        X[16] = self.knob_pos[3]  # v2
        X = X.reshape(1, -1)  # 1D to 2D
        
        Y = self.process(X)  # TODO: make the returned value always a 2D array
        self.inj_eff = Y[0]  # update the injection efficiency for monitoring the process
        if (not quiet) and self.verbose:
            self.render()
        
        return Y

    def reset(self):
        """
        Important: the observation must be a numpy array
        :return: (np.array) 
        """
        self.step_count = 0
        
        # Initialize the agent at the middle position of the knobs
        # and a x, xp, y, yp randomly offset dX
        self.knob_pos = 0.5 * np.ones(4)
        
        self.dX = 0.5 * np.ones(6)
        offset = 0.2 * np.random.rand(4) - 0.1
        self.dX[:4] += offset
        
        if self.verbose:
            print('\n' + '-' * 80)
            print(f'dX: {self.dX[0]:.4f}, {self.dX[1]:.4f}, {self.dX[2]:.4f}, {self.dX[3]:.4f}\n')
        
        # Calculate the reward and other states
        Y = self.get_reward(quiet=True)  # inj_eff, bpm1_x, bpm1_y, bpm2_x, ...
        
        obs = np.hstack((Y[1:], self.knob_pos))
        
        return obs

    def step(self, action):
        if action == self.H1_UP:
            self.knob_pos[0] += self.knob_step_size
        elif action == self.H1_DOWN:
            self.knob_pos[0] -= self.knob_step_size
        elif action == self.V1_UP:
            self.knob_pos[1] += self.knob_step_size
        elif action == self.V1_DOWN:
            self.knob_pos[1] -= self.knob_step_size
        elif action == self.H5_UP:
            self.knob_pos[2] += self.knob_step_size
        elif action == self.H5_DOWN:
            self.knob_pos[2] -= self.knob_step_size
        elif action == self.V2_UP:
            self.knob_pos[3] += self.knob_step_size
        elif action == self.V2_DOWN:
            self.knob_pos[3] -= self.knob_step_size
        else:
            raise ValueError(f'Received invalid action={action} which is not part of the action space')
            
        self.step_count += 1

        # Account for the knob position boundaries
        self.knob_pos = np.clip(self.knob_pos, 0, 1)
        Y = self.get_reward()

        obs = np.hstack((Y[1:], self.knob_pos))
        reward = Y[0]
        
        # Do we have a high enough injection efficiency already?
        done = bool((reward > 0.86) or (self.step_count == self.max_steps))

        # Optionally we can pass additional info, we are not using that for now
        info = {}

        return obs, reward, done, info

    def render(self, mode='console'):
        if mode != 'console':
            pass
#             raise NotImplementedError()
        # Just print out the knob pos and the injection efficiency
#         print(f'dX: {self.dX[0]:.4f}, {self.dX[1]:.4f}, {self.dX[2]:.4f}, {self.dX[3]:.4f}')
        print(f'h1: {self.knob_pos[0]:.2f}, v1: {self.knob_pos[1]:.2f}, ' + \
              f'h5: {self.knob_pos[2]:.2f}, v2: {self.knob_pos[3]:.2f}, ' + \
              f'injection efficiency: {self.inj_eff:.4f}')

    def close(self):
        pass

#### Check the BTS env

In [96]:
from stable_baselines3.common.env_checker import check_env

In [127]:
env = BTSEnv(process, knob_step_size=0.1, max_steps=1000)
# If the environment don't follow the interface, an error will be thrown
check_env(env, warn=True, skip_render_check=False)

h1: 0.50, v1: 0.60, h5: 0.50, v2: 0.50, injection efficiency: 0.0000


The following one is a manual version:

In [128]:
env = BTSEnv(process, knob_step_size=0.1, max_steps=1000, verbose=True)
obs = env.reset()
n_steps = 10
for _ in range(n_steps):
    # Random action
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)


--------------------------------------------------------------------------------
dX: 0.4797, 0.5616, 0.4959, 0.4169

h1: 0.50, v1: 0.50, h5: 0.50, v2: 0.40, injection efficiency: 0.9080
h1: 0.50, v1: 0.50, h5: 0.60, v2: 0.40, injection efficiency: 0.4130
h1: 0.50, v1: 0.40, h5: 0.60, v2: 0.40, injection efficiency: 0.0000
h1: 0.50, v1: 0.40, h5: 0.60, v2: 0.30, injection efficiency: 0.0000
h1: 0.50, v1: 0.30, h5: 0.60, v2: 0.30, injection efficiency: 0.0000
h1: 0.50, v1: 0.30, h5: 0.50, v2: 0.30, injection efficiency: 0.0000
h1: 0.50, v1: 0.30, h5: 0.60, v2: 0.30, injection efficiency: 0.0000
h1: 0.40, v1: 0.30, h5: 0.60, v2: 0.30, injection efficiency: 0.0000
h1: 0.40, v1: 0.30, h5: 0.70, v2: 0.30, injection efficiency: 0.0000
h1: 0.40, v1: 0.30, h5: 0.70, v2: 0.20, injection efficiency: 0.0000


### Train!

In [146]:
import os

from stable_baselines3 import PPO
# from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

from tqdm.auto import trange, tqdm

#### Instantiate the env

In [148]:
# Instantiate the env
env = BTSEnv(process, knob_step_size=0.1, max_steps=100)

model = PPO('MlpPolicy', env, verbose=0)

In [149]:
# Create log dir
log_dir = 'rl-log/'
os.makedirs(log_dir, exist_ok=True)

# Wrap the environment
env = Monitor(env, log_dir)

#### Evaluate the model before the training

In [150]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, render=False)

In [151]:
mean_reward, std_reward

(6.374599999999999, 16.34639357901308)

#### Train the RL model

In [152]:
# Train the agent for 1000 steps
model.learn(total_timesteps=1000)

#### Evaluate the trained model

In [153]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, render=False)

In [154]:
mean_reward, std_reward

(1.0887, 0.15930163213225407)

## Backlogs

In [155]:
def evaluate(model, num_episodes=100):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    env = model.get_env()
    all_episode_rewards = []
    for i in trange(num_episodes):
        episode_rewards = []
        done = False
        obs = env.reset()
        while not done:
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, reward, done, info = env.step(action)
            episode_rewards.append(reward[0])

        all_episode_rewards.append(episode_rewards)

    return all_episode_rewards

In [156]:
# Random Agent, before training
rewards = evaluate(model, num_episodes=10)

  0%|          | 0/10 [00:00<?, ?it/s]

In [157]:
rewards

[[0.0,
  0.0,
  0.0,
  0.0,
  0.27,
  0.241,
  0.27,
  0.241,
  0.521,
  0.422,
  0.803,
  0.803,
  0.422,
  0.331,
  0.664,
  0.664,
  0.477,
  0.283,
  0.152,
  0.202,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.152,
  0.246,
  0.17,
  0.1,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.069,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.156,
  0.0,
  0.422,
  0.0,
  0.0,
  0.018,
  0.0,
  0.018,
  0.0,
  0.002,
  0.001,
  0.001,
  0.2,
  0.414,
  0.576,
  0.697,
  0.765,
  0.697,
  0.006,
  0.007,
  0.0,
  0.0,
  0.354,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.339, 0.389, 0.884],
 [0.885],
 [0.91],
 [0.555, 0.904],
 [0.881],
 [0.907],
 [0.181, 0.899],
 [0.908],
 [0.89]]

---

In [18]:
a = b = 17

In [19]:
a, b

(17, 17)

In [7]:
low = np.array([-100] * 18 + [0] * 4)
high = np.array([100] * 18 + [1] * 4)

In [64]:
low, high

(array([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100,    0,    0,    0,    0]),
 array([100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100,   1,   1,   1,   1]))

---

In [33]:
X = 0.5 * np.ones((1, 22))

In [46]:
process(X)

array([ 0.908     ,  0.04201704,  0.01295851,  0.11609385,  0.01590106,
        0.16815135,  0.02435282,  0.10337124,  0.0113079 , -0.00438291,
       -0.00856344, -0.01582888, -0.02946351, -0.0279356 , -0.01365623,
       -0.02691261, -0.00738051, -0.02849029,  0.00383037])

In [12]:
np.hstack((np.ones(3), np.zeros(2)))

array([1., 1., 1., 0., 0.])

In [60]:
X[0, 4] = 0.5001

In [61]:
process(X)

array([ 0.905     ,  0.02054788,  0.01209201,  0.0740073 ,  0.00985236,
       -0.02654498,  0.0073793 , -0.24051572, -0.00655828, -0.33538095,
       -0.01576651, -0.12284825,  0.00647419,  0.10363903,  0.0116303 ,
        0.0881582 ,  0.03076752,  0.08574401,  0.06793308])

In [44]:
X[0, 6] = 0.6

In [49]:
process(X)

teeport: already linked


array([ 0.908     ,  0.04201704,  0.01295851,  0.11609385,  0.01590106,
        0.16815135,  0.02435282,  0.10337124,  0.0113079 , -0.00438291,
       -0.00856344, -0.01582888, -0.02946351, -0.0279356 , -0.01365623,
       -0.02691261, -0.00738051, -0.02849029,  0.00383037])

In [11]:
process(X + 0.01 * np.random.rand(1, 22))

array([  0.        ,  -1.42496657,  -0.0268286 ,  -2.75821768,
        -0.34319369, -13.36452708,  -1.00140158, -23.94383618,
        -1.06070593, -23.16685735,  -0.42571632,  -7.27960197,
         2.25141362,   9.61266597,   1.64698367,   8.3937019 ,
         2.57812122,   8.32116593,   4.44035147])

---

In [13]:
import gym

from stable_baselines3 import A2C

env = gym.make('CartPole-v1')

model = A2C('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

obs = env.reset()
for i in range(1000):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
        obs = env.reset()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 25.7     |
|    ep_rew_mean        | 25.7     |
| time/                 |          |
|    fps                | 314      |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.679   |
|    explained_variance | 0.0612   |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 1.95     |
|    value_loss         | 9.54     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 27.7     |
|    ep_rew_mean        | 27.7     |
| time/                 |          |
|    fps                | 336      |
|    iterations         | 200      |
|    time_elapsed